# scraping wikipedia page using Pandas

####    1. install prerequesties lxml, html5lib and beautifulsoup4

In [92]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [93]:

pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [227]:
from bs4 import BeautifulSoup
import pandas as pd



import numpy as np
import requests

import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [95]:
### put the url in variable
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


wiki_page= requests.get(url).text

soup= BeautifulSoup(wiki_page, "html5lib")

#print(soup.prettify())


#### extracting table from th webpage

In [96]:

postcode=[]
borough=[]
neighborhood=[]

rtable= soup.find('table', {'class':'wikitable sortable'})

for row in rtable.find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        postcode.append(cells[0].find(text = True))
        
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))




#### transforming our table int pandas data frame

In [97]:
df=pd.DataFrame(postcode, columns=['Postalcode'])
df['Borough']= borough
df['Neighborhood']= neighborhood

df.head(5)
#df.shape

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [98]:
# remove \n from our data frame

df=df.replace('\n', '', regex= True)

#### remove not assigned rows

In [99]:
# drop N\A values
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]


In [100]:
#combine neighborhoods that have same postal code area. 

df2= df.groupby('Postalcode').agg(lambda x: ','.join(x))


In [101]:

df2.shape

(103, 2)

In [102]:
# reset index and show the dataframe
df2=df2.reset_index()
df2.head()


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Question 2
####  Adding latitude and longitude to the data

In [103]:
geo_df=pd.read_csv('https://cocl.us/Geospatial_data')

In [104]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [139]:
geo_df.shape

(103, 3)

In [105]:
#change the name of so we can merge it later 
geo_df.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

# merge geo data with our data frame

merge_df=pd.merge(geo_df, df2, on='Postalcode' )


In [106]:
geo_data=merge_df[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]


In [144]:
geo_data.shape

(103, 5)

In [107]:
geo_data.tail(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [108]:
geo_data.shape

(103, 5)

### Question 3


In [137]:
toronto_data= geo_data[geo_data['Borough'].str.contains('Toronto', regex=False)]


In [138]:
toronto_data.shape

(39, 5)

In [111]:
toronto_data.tail(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
85,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
87,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558


In [32]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [112]:
from geopy.geocoders import Nominatim
import folium

In [145]:
address= 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [41]:
tor_map= folium.Map(location=[latitude , longitude], zoom_start=11)

# add markers to map
for lat, long, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_capacity=0.5,
        parse_html=False).add_to(tor_map)
    
tor_map


In [118]:
CLIENT_ID='QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ'

CLIENT_SECRET='CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM'

VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ
CLIENT_SECRET:CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM


##  Explore Neighborhoods in Toronto
    

In [181]:
def getNearbyVenues(names, latitudes, longitudes):

    radius=500
    limit=100
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        limit)
        results=requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
        
        
    
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood',
                           'Neighborhood Latitude', 
                           'Neighborhood Longitude', 
                           'Venue',
                           'Venue Latitude',
                           'Venue Longitude',
                           'Venue Category']
    return(nearby_venues)
    
    

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [182]:
toronto_venues=getNearbyVenues(names= toronto_data['Neighborhood'],
                                latitudes=toronto_data['Latitude'], 
                               longitudes=toronto_data['Longitude']
                              )



In [183]:
toronto_venues.shape


(1606, 7)

In [184]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [185]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [186]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## 3. Analyze Each Neighborhood


In [194]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 

toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'])

toronto_onehot.head()


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [195]:
toronto_onehot.shape

(1606, 233)

In [196]:
toronto_grouped= toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing Centre,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.013514,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013514,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.027027
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [198]:
toronto_grouped.shape

(39, 233)

In [204]:
def return_most_common_venues(row, num_top_venues):
    row_categories= row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending= False)
    
    return(row_categories_sorted.index.values[0: num_top_venues])

In [206]:


num_top_venues=10

indicators = ['st', 'nd', 'rd']

columns= ['Neighborhood']

for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} most common venue '.format(i+1, indicators[i]))
    except:
        columns.append('{}Th common vanue '.format(i+1))
        
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']= toronto_grouped['Neighborhood']


for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1 :]= return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4Th common vanue,5Th common vanue,6Th common vanue,7Th common vanue,8Th common vanue,9Th common vanue,10Th common vanue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Beer Bar,Diner,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Grocery Store,Restaurant,Bar,Intersection,Bakery,Stadium
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Plane,Airport Food Court
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Restaurant,Ice Cream Shop,Bar,Salad Place,Thai Restaurant


In [210]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering= toronto_grouped.drop('Neighborhood', 1)

kmeans= KMeans(n_clusters= kclusters, random_state=0 ).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [224]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_ )

toronto_merged= toronto_data

toronto_merged= toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on ='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Cluster Label,1st most common venue,2nd most common venue,3rd most common venue,4Th common vanue,5Th common vanue,6Th common vanue,7Th common vanue,8Th common vanue,9Th common vanue,10Th common vanue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0,Asian Restaurant,Health Food Store,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Department Store,Donut Shop
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Juice Bar
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,0,Sandwich Place,Fast Food Restaurant,Pizza Place,Pet Store,Pub,Liquor Store,Burrito Place,Fish & Chips Shop,Italian Restaurant,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,1,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [233]:
# Visualize the resulting clusters

map_clusters=folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x=np.arange(kclusters)
ys=[i + x + (i*x)**2 for i in np.arange(kclusters)]

colors_array=cm.rainbow(np.linspace(0,1, len(ys)))

rainbow=[colors.rgb2hex(i) for i in colors_array]


# add markers to the map

markers_colors=[]

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],
                                  toronto_merged['Longitude'],
                                 toronto_merged['Neighborhood'],
                                 toronto_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+ ' cluster '+ str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_capacity=0.7).add_to(map_clusters)

map_clusters



